# Pipeline-DeBERTa Training for DimABSA 2026

**Subtask 2**: Dimensional Aspect Sentiment Triplet Extraction (DimASTE)

**Model**: Pipeline-based approach with DeBERTa-v3-base encoder

---

## Setup
- GPU: T4 or P100 recommended
- Training time: ~2-3 hours for both domains
- Output: Trained models + predictions

## 1. Install Dependencies

In [ ]:
!pip install -q transformers torch sentencepiece protobuf spacy
!python -m spacy download en_core_web_sm

## 2. Clone/Upload Code

**Option A**: Upload Pipeline-DeBERTa folder as Kaggle dataset

**Option B**: Clone from GitHub (if you pushed it)

In [ ]:
# If uploaded as Kaggle dataset, copy to working directory
import shutil
import os

# Adjust path to your Kaggle dataset
# !cp -r /kaggle/input/pipeline-deberta/* /kaggle/working/

# Or if cloning from GitHub:
# !git clone https://github.com/YOUR-USERNAME/dimabsa-pipeline-deberta.git
# %cd dimabsa-pipeline-deberta

# For now, assume code is in /kaggle/working/Pipeline-DeBERTa
%cd /kaggle/working

## 3. Download Dataset

In [ ]:
# Download DimABSA 2026 dataset
!mkdir -p data/track_a/subtask_2/eng

# Restaurant data
!wget -q https://raw.githubusercontent.com/DimABSA/DimABSA2026/main/task-dataset/track_a/subtask_2/eng/eng_restaurant_train_alltasks.jsonl \
    -O data/track_a/subtask_2/eng/eng_restaurant_train_alltasks.jsonl
!wget -q https://raw.githubusercontent.com/DimABSA/DimABSA2026/main/task-dataset/track_a/subtask_2/eng/eng_restaurant_dev_task2.jsonl \
    -O data/track_a/subtask_2/eng/eng_restaurant_dev_task2.jsonl

# Laptop data
!wget -q https://raw.githubusercontent.com/DimABSA/DimABSA2026/main/task-dataset/track_a/subtask_2/eng/eng_laptop_train_alltasks.jsonl \
    -O data/track_a/subtask_2/eng/eng_laptop_train_alltasks.jsonl
!wget -q https://raw.githubusercontent.com/DimABSA/DimABSA2026/main/task-dataset/track_a/subtask_2/eng/eng_laptop_dev_task2.jsonl \
    -O data/track_a/subtask_2/eng/eng_laptop_dev_task2.jsonl

print("✓ Dataset downloaded")
!ls -lh data/track_a/subtask_2/eng/

## 4. Verify Setup

In [ ]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 5. Train Restaurant Domain

**Dataset**: 2,284 training samples, 200 dev samples

**Time**: ~30-45 minutes

In [ ]:
!python run_task2\&3_trainer_multilingual.py \
  --task 2 \
  --domain res \
  --language eng \
  --train_data data/track_a/subtask_2/eng/eng_restaurant_train_alltasks.jsonl \
  --infer_data data/track_a/subtask_2/eng/eng_restaurant_dev_task2.jsonl \
  --bert_model_type microsoft/deberta-v3-base \
  --mode train \
  --epoch_num 3 \
  --batch_size 8 \
  --learning_rate 1e-3 \
  --tuning_bert_rate 1e-5 \
  --inference_beta 0.9 \
  --gpu True

## 6. Check Restaurant Results

In [ ]:
import json

# Check predictions
pred_file = "tasks/subtask_2/pred_eng_restaurant.jsonl"
with open(pred_file, 'r') as f:
    predictions = [json.loads(line) for line in f]

print(f"Total predictions: {len(predictions)}")
print(f"\nFirst 3 predictions:")
for i, pred in enumerate(predictions[:3]):
    print(f"\n{i+1}. ID: {pred['ID']}")
    print(f"   Triplets: {len(pred['Triplet'])}")
    if pred['Triplet']:
        print(f"   Example: {pred['Triplet'][0]}")

## 7. Train Laptop Domain

**Dataset**: 4,076 training samples, 200 dev samples

**Time**: ~60-90 minutes

In [ ]:
!python run_task2\&3_trainer_multilingual.py \
  --task 2 \
  --domain lap \
  --language eng \
  --train_data data/track_a/subtask_2/eng/eng_laptop_train_alltasks.jsonl \
  --infer_data data/track_a/subtask_2/eng/eng_laptop_dev_task2.jsonl \
  --bert_model_type microsoft/deberta-v3-base \
  --mode train \
  --epoch_num 3 \
  --batch_size 8 \
  --learning_rate 1e-3 \
  --tuning_bert_rate 1e-5 \
  --inference_beta 0.9 \
  --gpu True

## 8. Check Laptop Results

In [ ]:
# Check predictions
pred_file = "tasks/subtask_2/pred_eng_laptop.jsonl"
with open(pred_file, 'r') as f:
    predictions = [json.loads(line) for line in f]

print(f"Total predictions: {len(predictions)}")
print(f"\nFirst 3 predictions:")
for i, pred in enumerate(predictions[:3]):
    print(f"\n{i+1}. ID: {pred['ID']}")
    print(f"   Triplets: {len(pred['Triplet'])}")
    if pred['Triplet']:
        print(f"   Example: {pred['Triplet'][0]}")

## 9. Validate Output Format

In [ ]:
def validate_predictions(pred_file):
    """Validate prediction format for Subtask 2"""
    with open(pred_file, 'r') as f:
        predictions = [json.loads(line) for line in f]
    
    errors = []
    for i, pred in enumerate(predictions):
        # Check required keys
        if 'ID' not in pred:
            errors.append(f"Line {i}: Missing 'ID'")
        if 'Triplet' not in pred:
            errors.append(f"Line {i}: Missing 'Triplet'")
            continue
        
        # Check triplet format
        for j, triplet in enumerate(pred['Triplet']):
            if 'Aspect' not in triplet:
                errors.append(f"Line {i}, Triplet {j}: Missing 'Aspect'")
            if 'Opinion' not in triplet:
                errors.append(f"Line {i}, Triplet {j}: Missing 'Opinion'")
            if 'VA' not in triplet:
                errors.append(f"Line {i}, Triplet {j}: Missing 'VA'")
            else:
                # Validate VA format
                va = triplet['VA']
                if '#' not in va:
                    errors.append(f"Line {i}, Triplet {j}: VA missing '#' separator")
                else:
                    parts = va.split('#')
                    if len(parts) != 2:
                        errors.append(f"Line {i}, Triplet {j}: VA should have exactly 2 values")
                    else:
                        try:
                            v, a = float(parts[0]), float(parts[1])
                            if not (1.0 <= v <= 9.0):
                                errors.append(f"Line {i}, Triplet {j}: Valence {v} out of range [1,9]")
                            if not (1.0 <= a <= 9.0):
                                errors.append(f"Line {i}, Triplet {j}: Arousal {a} out of range [1,9]")
                        except ValueError:
                            errors.append(f"Line {i}, Triplet {j}: VA values not numeric")
    
    if errors:
        print(f"❌ Found {len(errors)} errors:")
        for err in errors[:10]:  # Show first 10
            print(f"  - {err}")
        if len(errors) > 10:
            print(f"  ... and {len(errors)-10} more")
        return False
    else:
        print(f"✅ All {len(predictions)} predictions valid!")
        return True

print("Validating Restaurant predictions:")
validate_predictions("tasks/subtask_2/pred_eng_restaurant.jsonl")

print("\nValidating Laptop predictions:")
validate_predictions("tasks/subtask_2/pred_eng_laptop.jsonl")

## 10. Download Results

In [ ]:
# Create zip file with all results
!mkdir -p results
!cp model/*.pth results/
!cp tasks/subtask_2/*.jsonl results/
!cp log/*.log results/

!zip -r pipeline_deberta_results.zip results/

print("✓ Results packaged")
print("\nDownload: pipeline_deberta_results.zip")
print("\nContents:")
!ls -lh results/

## 11. Summary

In [ ]:
import json

print("=" * 60)
print("TRAINING SUMMARY")
print("=" * 60)

# Restaurant
with open("tasks/subtask_2/pred_eng_restaurant.jsonl", 'r') as f:
    res_preds = [json.loads(line) for line in f]
    res_triplets = sum(len(p['Triplet']) for p in res_preds)

print(f"\n📊 Restaurant Domain:")
print(f"  - Predictions: {len(res_preds)}")
print(f"  - Total triplets: {res_triplets}")
print(f"  - Avg triplets/sample: {res_triplets/len(res_preds):.2f}")
print(f"  - Model: model/task2_res_eng.pth")

# Laptop
with open("tasks/subtask_2/pred_eng_laptop.jsonl", 'r') as f:
    lap_preds = [json.loads(line) for line in f]
    lap_triplets = sum(len(p['Triplet']) for p in lap_preds)

print(f"\n💻 Laptop Domain:")
print(f"  - Predictions: {len(lap_preds)}")
print(f"  - Total triplets: {lap_triplets}")
print(f"  - Avg triplets/sample: {lap_triplets/len(lap_preds):.2f}")
print(f"  - Model: model/task2_lap_eng.pth")

print(f"\n✅ Training Complete!")
print(f"\n📦 Download: pipeline_deberta_results.zip")
print("=" * 60)